# Test webcam bounding box detection in notebook

Use OpenCV to read webcam feed, use tensorflow do perform bounding box detection and use ipywidgets to display videostream.

In [ ]:
import os
import sys
import time
import tarfile
import zipfile
import six.moves.urllib as urllib
from collections import defaultdict
from io import StringIO
import threading

import numpy as np
import tensorflow as tf
import cv2

from ipywidgets import widgets
from IPython.display import display

from test_model import get_graph_tensors, process_image

In [ ]:
# What model to download.
PATH_TO_CKPT = 'exported_graphs/frozen_inference_graph.pb'

In [ ]:
class WebcamBoundingBoxThread(threading.Thread):
    """
    Background thread to read image from webcam, detect objects, and update
     interactive display.
    """
    def __init__(self, interactive_img, interactive_framerate_text, model_path):
        super(WebcamBoundingBoxThread, self).__init__()
        self._stop_event = threading.Event()
        self.model_path = model_path
        self.interactive_img = interactive_img
        self.interactive_framerate_text = interactive_framerate_text
        
    def setup(self):
        """
        Setup tensorflow graph and webcam connection.
        """
        # Setup Tensorflow detector
        (self.detection_graph, self.tensor_dict, 
         self.input_image_tensor) = get_graph_tensors(self.model_path)
        # Setup camera capture
        self.camera = cv2.VideoCapture(0)

    def stop(self):
        """
        Stop thread.
        """
        self._stop_event.set()

    def stopped(self):
        """
        True iff tread is stopped.
        """
        return self._stop_event.is_set()
    
    def process_frame(self, frame):
        """
        Process a single frame for bounding box detection.
        """
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = process_image(
            image, self.session, self.tensor_dict, self.input_image_tensor)
        return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    def run(self):
        """
        Start thread.
        Creates loop that reads frame from webcam, detects bounding boxes,
         and updates interactive image until thread is stopped.
        """
        self.setup()
        is_capturing = self.camera.isOpened()
        start_time  = time.time()
        with self.detection_graph.as_default():
            self.session = tf.Session()
            with self.session as sess:
                while is_capturing and not self.stopped():
                    start_time  = time.time()
                    is_capturing, frame = self.camera.read()
                    processed_frame = self.process_frame(frame)
                    self.interactive_img.value = cv2.imencode('.png', processed_frame)[1].tostring()
                    self.interactive_framerate_text.value = '{:.2f}'.format(1/(time.time() - start_time))
        self.camera.release()

In [ ]:
print('Start')
# Create interactive image
interactive_img = widgets.Image(
    value=b'',
    format='png',
    width=800,
    height=600,
)

# Create interactive text to display framerate
interactive_framerate_text = widgets.Text(
    value='0',
    placeholder='0',
    description='Fps:',
    disabled=False
)


# Create thread to update interactive image with webcam
print('Setup thread')
thread = WebcamBoundingBoxThread(
    interactive_img, interactive_framerate_text, PATH_TO_CKPT)
thread.daemon = True
print('Display image')
display(interactive_img)
display(interactive_framerate_text)
print('Starting thread')
thread.start()


# Stop thread upon exit
print('Running loop')
while True:
    try:
        time.sleep(1)
    except:
        thread.stop()
        thread.join()
        break

        
print('Finish')